In [1]:
from brightway2 import *
from rower import *

In [2]:
projects.set_current("RoW generation")

In [3]:
if not len(databases):
    bw2setup()

Creating default biosphere

Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applied 2 strategies in 0.01 seconds


Writing activities to SQLite3 database:
0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 05/25/2018 10:09:22
  Finished: 05/25/2018 10:09:22
  Total time elapsed: 00:00:00
  CPU %: 97.40
  Memory %: 0.88
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 4 strategies in 1.30 seconds
Wrote 718 LCIA methods with 178008 characterization factors
Creating core data migrations



In [4]:
dbs = [
    ("/Users/cmutel/Documents/LCA/Ecoinvent/3.3/cutoff/datasets", "3.3 cutoff"),
    ("/Users/cmutel/Documents/LCA/Ecoinvent/3.3/apos/datasets", "3.3 apos"),
    ("/Users/cmutel/Documents/LCA/Ecoinvent/3.3/consequential/datasets", "3.3 consequential"),    
    ("/Users/cmutel/Documents/LCA/Ecoinvent/3.4/cutoff/datasets", "3.4 cutoff"),
    ("/Users/cmutel/Documents/LCA/Ecoinvent/3.4/apos/datasets", "3.4 apos"),
    ("/Users/cmutel/Documents/LCA/Ecoinvent/3.4/consequential/datasets", "3.4 consequential"),    
]

In [5]:
for x, y in dbs:
    ei = SingleOutputEcospold2Importer(x, y)
    ei.apply_strategies()
    assert not ei.statistics()[2]
    ei.write_database()

Extracting XML data from 13330 datasets
Extracted 13330 datasets in 52.16 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre34
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
2 exchanges couldn't be linked and were deleted. See the logfile for details:
	/Users/cmutel/Library/Logs/Brightway3/RoW-generation.3c2f62ecccd4625845dbffb32aef3443/Ecospold2-import-error.yttzM4.log
Applying st

Writing activities to SQLite3 database:
0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:34


Title: Writing activities to SQLite3 database:
  Started: 05/25/2018 10:57:34
  Finished: 05/25/2018 10:58:08
  Total time elapsed: 00:00:34
  CPU %: 94.30
  Memory %: 5.38
Created database: 3.4 consequential


In [3]:
rows = set()

for name in databases:
    if name == "biosphere3":
        continue

    rower = Rower(name)
    data = rower._reformat_rows(rower._load_groups_sqlite())
    rows.update(set(data))

In [4]:
len(rows)

223

In [5]:
rows_master = {"RoW_{}".format(i): j for i, j in enumerate(sorted(rows))}
master_reversed = {j: i for i, j in rows_master.items()}

In [8]:
dp = RowerDatapackage(os.path.join(DATAPATH, "ecoinvent generic"))

In [9]:
dp.write_data("ecoinvent generic", rows_master)

In [6]:
for name in databases:
    if name == "biosphere3":
        continue
    r = Rower(name)    
    r.existing = master_reversed
    labelled, user_rows = r.define_RoWs()
    assert not user_rows
    print(name, len(labelled))
    rows_used = {i: j for i, j in rows_master.items() if i in labelled}
    dp = RowerDatapackage(os.path.join(DATAPATH, "ecoinvent " + name))
    dp.write_data("ecoinvent " + name, rows_used, labelled)

3.3 cutoff 181
3.3 apos 182
3.3 consequential 176
3.4 cutoff 186
3.4 apos 189
3.4 consequential 181
